In [1]:
!pip install split-folders

In [4]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=925dc5517cf0881077c70cff4771b7961392609c422303d9d95130944c388da3
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [9]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import transforms
import os
from PIL import Image
import torch
import ssl
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import splitfolders
import csv
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from efficientnet_pytorch import EfficientNet
import warnings

warnings.filterwarnings(
    action='ignore')


def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

ssl._create_default_https_context = ssl._create_unverified_context

if __name__ == '__main__':
    # Define the device to be used for training
    device = torch.device("cpu")

    # Set up the transform to resize and normalize the images
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]),
    ])
     
   # loading the temp.zip and creating a zip object 
  # with ZipFile("C:\\Users\\sai mohan pulamolu\\\ 
   #         Desktop\\geeks_dir\\temp\\temp.zip", 'r') as zObject: 
  
    # Extracting all the members of the zip  
    # into a specific location. 
   # zObject.extractall( 
    #    path="C:\\Users\\sai mohan pulamolu\\Desktop\\geeks_dir\\temp") 
    # Update input folder and output folder paths
    input_folder = r"/kaggle/input/dataset-small/IMAGES_HighPassFilter_Small"
    output_folder = r"/kaggle/working/images_30_output"
    

    ### Uncomment only for first time. once data is splitted into train and validation, comment it out
    splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.2), group_prefix=None)

    # Create datasets for the training and testing sets
    train_dataset = torchvision.datasets.ImageFolder(output_folder + '/train', transform=transform)
    val_dataset = torchvision.datasets.ImageFolder(output_folder + '/val', transform=transform)
    train_size = len(train_dataset)
    val_size = len(val_dataset)

    # Create the data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True,num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=25, shuffle=True,num_workers=4)
    list_of_classes = os.listdir(r"/kaggle/working/images_30_output/train")
    print(list_of_classes)
    classes = list(train_dataset.class_to_idx.keys())
    classes.sort()


    # Define the VGG model
    model_name = 'efficientnet-b0'  # You can change this to other versions (b1, b2, etc.)
    model = EfficientNet.from_pretrained(model_name)
    num_features = model._fc.in_features
    model._fc = nn.Linear(num_features, len(list_of_classes))

  

    ## uncomment for CPU
    model = model.to(device)

    # Define the loss function and optimizer
    
    
    criterion = nn.CrossEntropyLoss()
    START_LR = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=START_LR)
    model = model.to(device)
    criterion = criterion.to(device)

    ### uncomment for GPU
    #model.cuda()
        # Train the model
    for epoch in range(10):
        ### uncomment for GPU
        #torch.cuda.empty_cache()
        print('Epoch {}/{}'.format(epoch + 1, 10))
        print('-' * 10)

        running_loss = 0
        running_corrects = 0

        model.train()
        predictions = []
        true_labels = []
        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        report_dict = classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_pd = pd.DataFrame(report_dict)
        report_pd.to_csv('training-classification-epoch' + str(epoch + 1) + '.csv')
        cnf_matrix = confusion_matrix(true_labels, predictions)
        df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])
        df_cm.to_csv('confusion-matrix-train-epoch' + str(epoch + 1) + '.csv')
        #acc = matrix.diagonal()/matrix.sum(axis=1)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        ACC = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        PPV = TP/(TP+FP)
        print("accuracy for all classes in train phase", ACC)
        print("recall for all classes in train phase", TPR)
        print("precision for all classes in train phase", PPV)
        
        pd.DataFrame(ACC, columns=['Accuracy']).to_csv('accuracy-train-epoch' + str(epoch + 1) + '.csv')
        print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # Delete data to clear GPU memory
        del outputs
        del preds
        del labels
        del inputs
        torch.cuda.empty_cache()
        
        # Validation phase
        running_loss = 0
        running_corrects = 0
        model.eval()  # set the model to evaluation mode
        predictions = []
        true_labels = []
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / val_size
        epoch_acc = running_corrects.double() / val_size
       
        #classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_dict = classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_pd = pd.DataFrame(report_dict)
        report_pd.to_csv('val-classification-epoch' + str(epoch + 1) + '.csv')
        #matrix = confusion_matrix(true_labels, predictions)
        cnf_matrix = confusion_matrix(true_labels, predictions)
        df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])
        df_cm.to_csv('confusion-matrix-val-epoch' + str(epoch + 1) + '.csv')
        #acc = matrix.diagonal()/matrix.sum(axis=1)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        ACC = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        PPV = TP/(TP+FP)
        pd.DataFrame(ACC, columns=['Accuracy']).to_csv('accuracy-val-epoch' + str(epoch + 1) + '.csv')
        # print("accuracy for all classes in validation phase", ACC)
        # print("recall for all classes in validation phase", TPR)
        # print("precision for all classes in validation phase", PPV)
        print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # Delete data to clear GPU memory
        del outputs
        del preds
        del labels
        del inputs
        torch.cuda.empty_cache()
                
        

    # Save the model
    torch.save(model, 'efficientnet_model.pth')

Copying files: 8689 files [00:09, 874.72 files/s] 
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


['fislov1', 'stusta1', 'yebere1', 'mouwag1', 'scrcha1', 'gobbun1', 'vimwea1', 'chucis1', 'afrthr1', 'hunsun2', 'whbtit5', 'mabeat1', 'afrgrp1', 'fotdro5', 'whbwhe3', 'grbcam1', 'hadibi1', 'crheag1', 'varsun2', 'kvbsun1', 'grwpyt1', 'gbesta1', 'bkctch1', 'wtbeat1', 'squher1', 'blnwea1', 'pabspa1', 'categr', 'blhgon1', 'whbwea1']


100%|██████████| 20.4M/20.4M [00:00<00:00, 56.0MB/s]


Loaded pretrained weights for efficientnet-b0
Epoch 1/10
----------


100%|██████████| 278/278 [19:22<00:00,  4.18s/it]


...

Val Loss: 0.4783 Acc: 0.8851
Epoch 10/10
----------


100%|██████████| 278/278 [16:44<00:00,  3.61s/it]


accuracy for all classes in train phase [0.99927944 0.99769419 0.99697363 0.9988471  0.99913532 0.99610895
 0.99956766 0.9988471  0.99899121 0.99351492 0.99841476 0.99697363
 0.99682951 0.99913532 0.99740597 0.99927944 0.99942355 0.99942355
 0.99942355 0.99985589 0.99927944 0.99812653 0.99956766 0.99755008
 0.99841476 0.99927944 0.99971177 0.9988471  0.9988471  0.99827064]
recall for all classes in train phase [0.97014925 0.98701299 0.97933884 0.96226415 0.91666667 0.99183673
 0.98863636 0.98571429 0.87804878 0.97965116 0.98015873 0.97641509
 0.98046875 0.96385542 0.97575758 0.88461538 0.86666667 0.94444444
 0.98192771 0.96       0.98181818 0.88571429 0.9        0.98666667
 0.90909091 0.95833333 0.96153846 0.97945205 0.95505618 0.9760479 ]
precision for all classes in train phase [0.95588235 0.97854077 0.97731959 0.96226415 0.98214286 0.98082745
 0.97752809 0.95833333 0.94736842 0.97681159 0.97628458 0.97411765
 0.9766537  0.96385542 0.98773006 0.92       1.         1.
 0.99390244 1.  

100%|██████████| 70/70 [01:31<00:00,  1.30s/it]

Val Loss: 0.4702 Acc: 0.8851
